# 第3回：DBSCAN & クラスタリングの実用上の問題

このHands-onでは下記の3種類のデータを用いて，DBSCANとK-meansの実用上の問題を体験します．
* 人工的な2次元データ1
* 中国は北京市のタクシーの位置データ
* 人工的な2次元データ2

Hands-onに先立って，必要なライブラリをインストールしてきます．

In [ ]:
# 地図可視化ライブラリのインストール
!pip install folium    

必要なライブラリも読み込んでおきます．

In [ ]:
# 表形式のデータを操作するためのライブラリ
import pandas as pd

# 行列計算をおこなうためのライブラリ
import numpy as np

# 機械学習用ライブラリsklearnのKmeansクラス
from sklearn.cluster import KMeans

# 機械学習用ライブラリsklearnのDBSCANクラス
from sklearn.cluster import DBSCAN

# 点同士の距離を計算する関数
from scipy.spatial import distance

# 地図の可視化ライブラリ
import folium

# グラフ描画ライブラリ
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns;
sns.set(style='ticks')
%matplotlib inline

# ファイルの操作用
import os


# 警告文を表示させないおまじない
import warnings
warnings.filterwarnings('ignore')


---

## 例題1: 人工データに対するDBSCANクラスタリング

DBSCANの効果やデータを分析してみましょう．

例題1で用いるデータは人工的に作られた8000個の2次元データです．データに特に意味はありません．
用いるデータは[University of Eastern Finlandの計算学部が公開しているデータセット](http://cs.joensuu.fi/sipu/datasets/)です．

以下のコードを実行して，``misc_df``変数にデータを読み込んでください．

In [ ]:
url = "http://cs.joensuu.fi/sipu/datasets/t4.8k.txt"
misc_df = pd.read_table(url, sep="\s+", header=0, names=['x', 'y'])

# 最初の10件のデータを表示
misc_df.head()

散布図でデータの分布を確認してみましょう．下記コードを実行してください．

In [ ]:
plt.scatter(misc_df.x, misc_df.y, alpha=0.1)
plt.show()

かなり人工的な分布ですが，DBSCANの特徴を体験するにはよい題材です．このデータを使って，DBSCANを体験してみましょう．

DBSCANの特徴のひとつは**クラスタ数を指定する必要がない**ことです．K-meansクラスタリングではクラスタリングを行うためにはクラスタ数を指定する必要がありました．DBSCANは**どれくらいデータが密集していたら，データの集まりをクラスタを見なす**かの閾値さえ与えれば，あとはクラスタ数は自ずと決まります．

では，上記データに対してまずK-meansを適用してみましょう．データを眺めるとクラスタ数は6あるように見えますので，`K=6`として下記K-meansのコードを実行してみましょう．

In [ ]:
# クラスタリングを実行
kmeans_model = KMeans(n_clusters=6, init='random')
kmeans_model.fit(misc_df)

# 結果を格納
kmeans_labels = kmeans_model.labels_

クラスタリングが完了したので，クラスタ毎にデータ点に色づけし可視化してみましょう．下記コードを実行してください．

In [ ]:
plt.scatter(
    misc_df.x, misc_df.y,
    c=kmeans_labels,
    alpha=0.3, # 透明度
    s=15, # マーカーのサイズ
    cmap="tab20" # カラーパレット（20色対応）
)

データが6箇所にクラスタリングされましたが，期待通りにクラスタリングできていません．本当は，「コの字」型のクラスタ，「T字」型のクラスタなどが抽出できてほしかったはずです．

今回のクラスタリングの結果をよく見ると，K-meansはある点を中心に楕円状にクラスタを抽出しているように見えます．これこそがK-meansの特徴のひとつです．授業でも話したように，K-meansはクラスタ生成のアルゴリズム上，`クラスタの形状が超球状`になるという性質があります．

おまけにK-meansはクラスタリング時にクラスタ数を指定する必要があります．今回はたまたまデータの全容を目視することができたため，クラスタ数を予想することができました．しかし，実際はデータは多次元であることが多く，データの全容を視覚的に捉えることは困難です．そのため，クラスタ数を決定するのが難しいこともしばしばあります．

上記のデータのように，
* クラスタ数を指定せずにクラスタリングしたい
* クラスタの形状が超球状以外のものにも対応したい

といったケースで有効な手法が`DBSCAN`です．早速同じデータにDBSCANを適用してみましょう．機械学習ライブラリ`scikit-learn`には`DBSCAN`も含まれています．すでに冒頭で`DBSCAN`クラスを読み込んでいますので，以下のコードを実行すればDBSCANでクラスタリングすることできます．

In [ ]:
# クラスタリングを実行
dbscan_model = DBSCAN(eps=10, min_samples=20) #min_samplesはminPtsのこと
dbscan_model.fit(misc_df)

# 結果を格納
dbscan_labels = dbscan_model.labels_

# クラスタリング結果を色づけして可視化
plt.scatter(
    misc_df.x,
    misc_df.y,
    c=dbscan_labels,
    alpha=0.3, # 透明度
    s=15, # マーカーのサイズ
    cmap="tab20" # 濃青はノイズ（-1）ラベル
)

DBSCANによって，「コの字」型のクラスタ，「T字」型のクラスタがうまく抽出できており，私たちの直感に合うクラスタリング結果が得られました．またクラスタの周辺にある点が「ノイズ（外れ値）」として識別できていることも確認できます．

上記コードでは，DBSCANのパラメータとして
* $\epsilon$ = 10（eps: クラスタ判定の距離）
* $minPts$ = 20（min_samples: コア点として判定するためのデータ個数）

を設定しました．授業でもお話ししたとおり，DBSCANはパラメータが非常に敏感で，適切な値を設定するのが難しいです．上記コードの`eps`，`min_samples`を適当に変更し，どのように結果が変わるか確認してみてください．


---
## 例題2: 中国は北京市のタクシーの移動データ

人工データばかり使っても面白くないので，今度は実データに対してDBSCANを適用してみましょう．今回用いるデータは，Microsoft Researchの研究グループが収集した「[北京市におけるタクシーの移動データ](https://www.microsoft.com/en-us/research/publication/t-drive-trajectory-data-sample/)」です．

このデータは，2008年の2月6日から2月8日にかけて，北京市のタクシー10357台の位置データを一定間隔で収集したもので，全部で1500万レコードのデータが含まれています．

出典は以下の通りです:
> Jing Yuan, Yu Zheng, Xing Xie, and Guangzhong Sun: "Driving with knowledge from the physical world". In The 17th ACM SIGKDD international conference on Knowledge Discovery and Data mining, KDD ’11, New York, NY, USA, 2011. ACM.


今回のHands-onで全データを扱うのは時間的に厳しいので，10357台のタクシーのうち，10台のタクシーの2月6日分のデータだけに焦点を当てて分析を行いましょう．下記のコードを何も考えずに実行してください．データがダウンロードされ，`taxi_df`という変数に分析で扱うデータが格納されます．


In [ ]:
# データをダウンロード
!curl -O https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/06.zip

# zipファイルを解凍
!unzip -q 06.zip -d taxi-dataset

# データをpandasデータフレームに格納
taxi_dfs = []
for i, filename in enumerate(os.listdir("taxi-dataset")):
    if i < 10:
        _df = pd.read_csv(
            'taxi-dataset/{}'.format(filename),
            names=['taxi_id', 'timestamp', 'longitude', 'latitude']
        ).assign(
            timestamp = lambda df: pd.to_datetime(df.timestamp)
        )

        taxi_dfs.append(_df)
    else:
        break

taxi_df = pd.concat(
    taxi_dfs
).drop_duplicates()

# 一時変数とダウンロードしたデータを削除
del taxi_dfs
!rm -rf 06.zip taxi-dataset

# データの表示
taxi_df

ダウンロードしたデータは全部で16892レコードあることが確認できます．また，データは
* `taxi_id`: タクシーのID
* `timestamp`: データが収集された日時
* `longitude`: タクシーの経度
* `latitude`: タクシーの緯度

で構成されていることが分かります．

データをX軸を経度，Y軸を緯度とする平面上にマッピングしてみましょう．下記コードを実行します．

In [ ]:
plt.scatter(taxi_df.longitude, taxi_df.latitude, alpha=0.1, s=5)
plt.show()

なんだかよく分かりませんよね... これを眺めていても面白くないので，データを地図にマッピングしてみましょう．下記コードを実行してください．

In [ ]:
# foliumで地図オブジェクトを作成
beijing_map = folium.Map(location=[39.916668, 116.383331], zoom_start=12)

# 地図上にデータをプロット
for latitude, longitude in taxi_df[['latitude', 'longitude']].values.tolist():
    circle = folium.Circle(
        radius=30,
        location=(latitude, longitude),
        fill=True,
        fill_opacity=0.2
    )
    circle.add_to(beijing_map)

# 地図を表示
beijing_map

北京市の地図上に，2月6日の10台のタクシーの移動データ（位置データ）がプロットされました．

今回のHands-onでは，このデータにDBSCANを適用することで，密な位置データをクラスタとして抽出します．タクシーは色々な所に移動して停車していますが，（時間を無視して）密なクラスタを抽出することで，タクシーがよく集まる場所，つまり
* タクシーがよく休憩しているエリア
* タクシーがよくお客を乗せるエリア（人が集まる場所）

を発見しましょう．

例題1でも述べたように，DBSCANはパラメータに敏感です．パラメータ設定にはデータに対する深い洞察と理解が必要となります．$\epsilon$（eps）パラメータはクラスタに含める点を見つけるために，コア点からの許容距離を定めるものでした．

地理座標系では地球上のどこにいるかによって緯度・経度1度あたりの距離は変わってきますが，ここでは**1度あたり85.379km**と考えることにしましょう．また，**100m**範囲内に別の点があれば，密になっていると考えることにしましょう．この仮定に基づくと，$\epsilon$パラメータは`eps=0.00117124`になります．$minPts$パラメータは，`min_samples=10`くらいだと言うことにしておきましょう．

では，下記コードを実行し，DBSCANを`taxi_df`に適用しましょう．


In [ ]:
# クラスタリングを実行
dbscan_model = DBSCAN(eps=0.00117124, min_samples=10) #min_samplesはminPtsのこと
dbscan_model.fit(taxi_df[['latitude', 'longitude']])

# 結果を格納
taxi_df['cluster'] = dbscan_model.labels_

cluster_num = len(set(taxi_df['cluster']))-1
print("クラスタ数: ", cluster_num)

DBSCANによって104個のクラスタが抽出されました．最後に各クラスタを地図上で確認してみましょう．下記のコードを実行します．色づけのためにコードが長くなっていますが，やっていることはクラスタ毎に色を指定しているだけです（ノイズ（外れ値）クラスタはほぼ透明の灰色に）．

In [ ]:
# foliumで地図オブジェクトを作成
beijing_map = folium.Map(location=[39.916668, 116.383331], zoom_start=12)

# カラーマップを用意
cmap = plt.cm.get_cmap('prism')

# 地図上にデータをプロット
for latitude, longitude, cluster_id in taxi_df[['latitude', 'longitude', 'cluster']].values.tolist():
    if cluster_id == -1:
        cluster_color = 'gray'
        opacity = 0.2
    else:
        cluster_color = mcolors.to_hex(cmap(cluster_id / cluster_num))
        opacity = 0.8
    
    circle = folium.Circle(
        radius=30,
        location=(latitude, longitude),
        fill=True,
        opacity=opacity,
        fill_opacity=0.2,
        color=cluster_color
    )
    circle.add_to(beijing_map)

# 地図を表示
beijing_map


---
## 例題3: エルボー法

K-meansのクラスタ数の決め方について体験してみましょう．

授業でもお話ししたように，K-meansはクラスタ数を指定する必要があります．クラスタリングによってデータを分割し，データ集合が性質もつ考察するためには，適切なクラスタ数を設定し，クラスタが細かくなりすぎないかつある程度意味をなすクラスタのサイズにデータを分割する必要があります．

K-meansにおけるクラスタ数の決め方の1つとして**エルボー法**があります．授業でも説明しましたが，エルボー法は
> クラスタ数を変化させながらクラスタ内誤差平方和（SSE）を計算し，SSEの値の変化パターンが大きく変わった際のクラスタ数を最適クラスタ数とする

というものです．例題3では，エルボー法を使って最適なクラスタ数を見つけてみましょう．

まずはデータを用意します．今回用いるデータは第2回の授業で用いた人工データです（[University of Eastern Finlandの計算学部が公開しているデータセット](http://cs.joensuu.fi/sipu/datasets/)）．以下のコードを実行して，``s1_df``変数にデータを読み込んでください．


In [ ]:
url = "http://cs.joensuu.fi/sipu/datasets/s1.txt"
s1_df = pd.read_table(url, sep="\s+", header=None, names=['x', 'y'])

# 最初の10件のデータを表示
s1_df.head(10)

データの傾向を見るために，XY平面にデータをプロットしてみましょう．
下記のコードを実行します．

In [ ]:
plt.scatter(s1_df.x, s1_df.y, alpha=0.3)
plt.show()

データを見る限り，クラスタは15個になりそうです．エルボー法は最適なクラスタ数を見つけることができるでしょうか．

エルボー法ではクラスタ内誤差平方和（SSE）を計算する必要がありますが，幸いなことに`scikit-learn`ライブラリはSSEを計算するための`inertia_`という関数を提供しています．以下のコードを実行し，クラスタ数を1から30まで変化させながらK-meansを実行してみましょう．

In [ ]:
# SSEを格納するための変数
SSEs = []

for K in range(1, 31):
    # クラスタリングを実行
    model = KMeans(n_clusters=K, init='random')
    model.fit(s1_df)
    
    # SSEの計算
    SSE = model.inertia_
    print("K=", K, ", SSE=", SSE)
    SSEs.append(SSE)

SSEの計算が完了しました．クラスタ数の増加に応じてSSEが減少していることが分かりますが，どこで減少傾向が大きく変化したかが分かりません．エルボー法を実行するために，以下のコードを実行してSSEの変化を可視化してみましょう．

In [ ]:
ax = sns.lineplot(x=[K for K in range(1, 31)], y=SSEs)
ax.set_xlim((0, 30))
ax.set_xlabel("Number of Clusters (K)")
ax.set_ylabel("SSE")
plt.show()

グラフ上で肘（エルボー）のようにガクっと変化しているところをはっきり見つけられませんが，大体`K=13〜15`あたりのところで減少傾向に変化が見られます（減少傾向が落ち着く）．よって，エルボー法では最適クラスタは`K=13〜15`くらいと判断することになります．

ここでは試しに`K=13`として，再度K-meansクラスタリングを実行してみましょう．

In [ ]:
# クラスタリングを実行
model = KMeans(n_clusters=13, init='random')
model.fit(s1_df)

# 結果を格納
labels = model.labels_

plt.scatter(
    s1_df.x, s1_df.y,
    c=labels,
    alpha=0.4, # 透明度
    s=40, # マーカーのサイズ
    cmap="tab20_r" # カラーパレット（20色対応）
)

データの全容を確認できる立場からすると`K=15`が妥当に思えますが，エルボー法でもまずまずのクラスタ数を見つけられたのではないでしょうか．


---
## 例題4: 次元の呪い

クラスタリングから離れて，「次元の呪い」を体験してみましょう．

授業でも説明したとおり，データの次元数が大きくなるとデータの特徴が正しく捉えることが難しくなります．これを**次元の呪い**と呼びます．次元の呪いは機械学習の精度を低下させる大きな要因になりますので，注意して対応する必要があります．

次元の呪いによって起きえる問題の1つとして，**超高次元空間ではデータ同士の類似性が判定できなくなる**ことが挙げられます．類似度計算は機械学習の肝であるので，データ同士の類似性を判定できなくなるのは致命的です．本例題では，この現象を体験してみましょう．

各次元の定義域が[-1, 1]であるN次元空間に，50個の点がランダムに散らばっている状況を考えます．例えば2次元空間であれば，以下のコードを実行することで，50個の点をランダムに生成することができます．

In [ ]:
M = np.random.uniform(-1, 1, (50, 2))
M

上のコードは結果を行列形式で返しています．行が各データ，列が各次元（1列目は1次元目の値，2列目は2次元目の値）に対応しています．これら50個について，点同士の（ユークリッド）距離を計算し，最大の距離と最小の距離を計算してみましょう．以下のコードを実行すると，50個の点同士の距離を一気に計算できます．

In [ ]:
dists = distance.pdist(M, metric='euclidean')
dists

点同士の距離が格納されたリストが得られました．最大の距離と最小の距離は，以下のコードで得られます．

In [ ]:
max_dist = max(dists)
min_dist = min(dists)
print("最大距離: ", max_dist)
print("最小距離: ", min_dist)

すでに説明したとおり，次元の呪いの問題の1つは，超高次元空間ではデータ間の類似性が判定できなくなるという問題でした．これは**「データ間の距離の最大値と最小値の差がゼロになる」**と言い換えることができます．上のデータの例の場合，最大距離と最小距離の差$\delta_{dist}$は

In [ ]:
max_dist - min_dist

となります．幸いなことに，この2次元のデータセットでは最大距離と最小距離の差はゼロにはなっていません．しかし，今回ランダムに発生させた点ではたまたま結果がゼロにならなかった可能性もあります．そこで，「ランダムに点を50個発生させ，点同士の距離の最大値と最小値の差$\delta_{dist}$を計算する」という操作を10回繰り返し，$\delta_{dist}$の平均値（$\bar{\delta_{dist}}$）を調べましょう．

この操作を行うコードは以下の通りです．

In [ ]:
deltas = []

# 10回計算
for i in range(10):
    M = np.random.uniform(-1, 1, (50, 2))
    dists = distance.pdist(M, metric='euclidean')
    delta = max(dists) - min(dists)
    deltas.append(delta)
    
# 最大距離と最小距離の差の平均
delta_mean = np.mean(deltas)
delta_mean

ゼロにはなりませんでした．（経験的にも明らかですが）2次元空間上では，類似度は問題にならないと言えそうです．では次元数を増やしていくとどうなるでしょうか．

ここでは次元数を$K$とします．$K$の値を1から2000（次元）まで変化させながら，上記の「最大距離と最小距離の差の平均値」を求める操作を行ってみましょう．

ここで一点だけ注意することがあります．各次元の定義域が[-1, 1]であるN次元空間における点同士のユークリッド距離がとりうる最大値についてです．
* 1次元空間では距離の最大値は$2=2\sqrt{1}$
* 2次元空間では距離の最大値は$2 \sqrt{1^2 + 1^2}=2\sqrt{2}$
* 3次元空間では距離の最大値は$2 \sqrt{1^2 + 1^2 + 1^2}=2\sqrt{3}$

の観察から分かるように，N次元空間における点同士の距離がとり得る最大値は$2\sqrt{N}$となり，次元数が増えるにつれてとりうる距離の最大値が大きくなります．ですので，単純に最大距離と最小距離の差の「絶対値」を調べても，相対的に差が小さくなっているのか分かりません．そこで，距離を$2\sqrt{N}$で割ることで，正規化された最大距離と最小距離の差がどうなるかを調べることにしましょう．

上記を踏まえた操作は以下となります（計算量が大きいので，結果が得られるまでに少し時間がかかります）．

In [ ]:
max_dim = 2000
data_num = 50

delta_means = []
for K in range(1, max_dim + 1):
    dist_deltas = []

    # 最大距離と最小距離の差の計算を10回
    for i in range(10):
        M = np.random.uniform(-1, 1, (data_num, K)) # K次元のデータをランダムに50個生成
        dists = distance.pdist(M, metric='euclidean') / (2 * (K ** 0.5)) # 正規化された距離
        dist_delta = max(dists) - min(dists)
        dist_deltas.append(dist_delta)

    # 最大距離と最小距離の差の平均
    delta_mean = np.mean(dist_deltas)
    delta_means.append(delta_mean)
    
# 折れ線グラフ上に結果を可視化
ax = sns.lineplot(x=list(range(1, max_dim + 1)), y=delta_means)
#ax.set_xlim((0, max_dim))
#ax.set_xlabel("Number of dimensions (K)")
#ax.set_ylabel("Distance delta")
plt.show()

次元数が大きくなると，最大距離と最小距離の差がゼロに近づいていくことが分かります．今回の実験ではN次元空間にデータがランダム（一様）に散らばっていることを想定していましたが，実際はデータの分布には何らかの偏りがあります．その場合でも，高次元空間ではデータ間の類似度・距離の扱いが難しいことには変わりはありません．特徴選択・次元圧縮などを行って，高次元データを低次元データに変換して機械学習を行う必要があります．